In [2]:
import torch
import torch.nn as nn
from torch.autograd import Variable

In [72]:
class LSTM(nn.Module):
  def __init__(self, in_size, hidden_size, batch_size,
               num_layers=1, dropout=.1,
               bidirectional=False, return_seq=True,
               batch_first=True, gpu=False,
               continue_seq=False):
    super(LSTM, self).__init__()
    self.in_size = in_size
    self.hidden_size = hidden_size
    self.num_layers = num_layers
    self.dropout = dropout
    self.batch_size = batch_size
    self.gpu = gpu
    self.bidirectional = bidirectional
    self.batch_first = batch_first
    self.return_seq = return_seq
    self.continue_seq = continue_seq
    if self.bidirectional:
      self.multi = 2
    else:
      self.multi = 1

    self.lstm = nn.LSTM(input_size=self.in_size,
                        hidden_size=self.hidden_size,
                        num_layers=self.num_layers,
                        bidirectional=self.bidirectional,
                        batch_first=self.batch_first,
                        dropout=self.dropout)

    if self.gpu:
      self.h0 = Variable(torch.zeros(self.multi*self.num_layers,
                                     self.batch_size,
                                     self.hidden_size).cuda())
      self.c0 = Variable(torch.zeros(self.multi*self.num_layers,
                                     self.batch_size,
                                     self.hidden_size).cuda())
    else:
      self.h0 = Variable(torch.zeros(2*self.num_layers,
                                     self.batch_size,
                                     self.hidden_size))
      self.c0 = Variable(torch.zeros(2*self.num_layers,
                                     self.batch_size,
                                     self.hidden_size))

  def reset_state(self, x):
    batch_size = x.size(0)
    if self.gpu:
      self.h0 = Variable(torch.zeros(self.multi*self.num_layers,
                                     batch_size,
                                     self.hidden_size).cuda())
      self.c0 = Variable(torch.zeros(self.multi*self.num_layers,
                                     batch_size,
                                     self.hidden_size).cuda())
    else:
      self.h0 = Variable(torch.zeros(2*self.num_layers,
                                     batch_size,
                                     self.hidden_size))
      self.c0 = Variable(torch.zeros(2*self.num_layers,
                                     batch_size,
                                     self.hidden_size))

  def forward(self, x):
    if self.continue_seq:
      h, (self.h0, self.c0) = self.lstm(x, (self.h0, self.c0))
    else:
      self.reset_state(x)
      h, _ = self.lstm(x, (self.h0, self.c0))

    if self.return_seq:
      pass
    else:
      h =  h[:,-1,:]

    return h


In [73]:
## 系列長15の5次元ベクトルをバッチサイズ32個分準備
batch_size = 32
seq_len = 15
features = 5

x = Variable(torch.randn(batch_size, seq_len, features))
print(x.size())

torch.Size([32, 15, 5])


In [74]:
## インプットの次元が5、隠れ状態（アウトプット）の次元が2のLSTM
## バッチサイズは隠れ状態とセル状態の初期値設定に必要
##（全てのバッチに対してそれぞれ異なる隠れ状態とセル状態を準備するため）

lstm = LSTM(in_size=5, hidden_size=2, batch_size=batch_size,
            return_seq=True, gpu=False, bidirectional=False)
print(lstm)

LSTM (
  (lstm): LSTM(5, 2, batch_first=True, dropout=0.1)
)


In [76]:
## 出力はバッチサイズ個だけあり、
## 系列長15のまま、LSTMのhidden_size次元に変換される

y_seq = lstm(x)
print(y_seq.size())

torch.Size([32, 15, 2])


In [78]:
## LSTMの最終出力（すなわち最後の時間の出力のみを取り出す）
## するとバッチサイズ個のhidden_size次元のデータが出てくる

lstm.return_seq=False
y_last = lstm(x)
print(y_last.size())

torch.Size([32, 2])


In [79]:
## 各系列データが入力されるごとに、内部状態は通常初期化されています。
## 
lstm.return_seq=True
lstm.continue_seq=False
x_data = torch.randn(1, 2 * 4, 5) ## バッチサイズ1, 系列長8, 特徴数5

x12 = Variable(x_data) ## 0~7までの系列

x1 = Variable(x_data[:, :4, :]) ## 0~3までの系列
x2 = Variable(x_data[:, 4:, :]) ## 4~7までの系列

In [80]:
## x12 に対するLSTMの系列出力
## バッチサイズは1で系列長8、特徴量2のデータが得られる
y12 = lstm(x12)
print(y12)

Variable containing:
(0 ,.,.) = 
  0.0137  0.1410
  0.0538 -0.1167
  0.2349 -0.0708
  0.0769 -0.2382
  0.0795 -0.0268
 -0.1107  0.1012
  0.0404 -0.1509
  0.0272 -0.1509
[torch.FloatTensor of size 1x8x2]



In [81]:
## x1に対するLSTMの系列出力とx2に対する系列出力
## y1は対する出力はy12の系列0~3と完全に一致する。
## y2はy12の系列4~7と一致しない（入力は同じなのに！）

y1 = lstm(x1)
y2 = lstm(x2)
print(y1)
print(y2)

Variable containing:
(0 ,.,.) = 
  0.0137  0.1410
  0.0538 -0.1167
  0.2349 -0.0708
  0.0769 -0.2382
[torch.FloatTensor of size 1x4x2]

Variable containing:
(0 ,.,.) = 
  0.0429  0.1554
 -0.1797  0.2660
  0.0274 -0.0814
  0.0260 -0.1408
[torch.FloatTensor of size 1x4x2]



In [95]:
## バックプロパゲーションの打ち切りは行うが
## 入力データの本来の繋がりを保ちたい場合


lstm.reset_state(x12)
y12 = lstm(x12)

lstm.continue_seq=True ## lstmが入力ごとに初期化することをやめ、前回の最終状態を保持する
lstm.reset_state(x1)
y1 = lstm(x1)
y2 = lstm(x2)
print(y12)
print(y1)
print(y2)

Variable containing:
(0 ,.,.) = 
  0.0137  0.1410
  0.0538 -0.1167
  0.2349 -0.0708
  0.0769 -0.2382
  0.0795 -0.0268
 -0.1107  0.1012
  0.0404 -0.1509
  0.0272 -0.1509
[torch.FloatTensor of size 1x8x2]

Variable containing:
(0 ,.,.) = 
  0.0137  0.1410
  0.0538 -0.1167
  0.2349 -0.0708
  0.0769 -0.2382
[torch.FloatTensor of size 1x4x2]

Variable containing:
(0 ,.,.) = 
  0.0795 -0.0268
 -0.1107  0.1012
  0.0404 -0.1509
  0.0272 -0.1509
[torch.FloatTensor of size 1x4x2]

